# Oanda Demo Trading Notebook

## Packages

Normal Packages

In [1]:
import numpy as np
import pandas as pd

import yaml
import json

import time
import datetime
import winsound
import collections

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

Oanda Packages

In [2]:
from oandapyV20 import API
import oandapyV20.endpoints.trades as trades
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts

import oandapyV20.definitions.pricing as defpricing
import oandapyV20.endpoints.instruments as instruments

## API Setup

Read from config file

In [3]:
config_file = 'config/access_token.yaml'

with open(config_file) as c_file:
    config = yaml.load(c_file)

access_token = config['oanda_demo_account']['token']
accountID = config['oanda_demo_account']['account_id']

api = API(access_token = access_token)

## Functions

In [4]:
# Get timestamp of the price and segregate it
def get_date_time(resp):
    
    time_stamp = resp['time']
    date_val, full_time = time_stamp.split(sep = 'T')
    time_val, time_fraction = full_time.split(sep = '.')
    
    return(date_val, time_val, time_fraction)

In [5]:
# Get bid and ask prices
def get_prices(resp):
    
    bid_price = float(resp['bids'][0]['price'])    
    ask_price = float(resp['asks'][0]['price'])
    spread = ask_price - bid_price
    tick_price = (ask_price + bid_price) / 2
    
    return(bid_price, ask_price, spread, tick_price)

In [6]:
# Terminate connection
def terminate_connection():
    try:
        r.terminate(message = "maxrecs records received")
    except:
        pass

In [7]:
# Restrict prices to 4 decimal places (pips)
def restrict_to_pips(num):
    numstring = str(num)
    num = float(numstring[:numstring.find('.')+5])
    return(num)

In [8]:
# Add prices into chunks for direction identification
def get_chunks(i, val):
    global chunk_size
    global small_list
    global size_flag
    global change_position
    
    
    if len(small_list) < chunk_size: #Keep adding prices until we reach chunk size
        small_list.append(val)  
               
    if size_flag == 'reached_chunk_size':        
        if small_list[-1] != val: 
            small_list.popleft()   #Remove the left most (first) price
            small_list.append(val) #Add current value to right most position

                
        change_position -= 1  #Price allocation in small list happens right to left so change position goes from chunk size to zero
        if change_position < 0: 
            change_position = chunk_size - 1 #If position reaches zero, it needs to be reset to chunk size - 1 (for index position)

            
    if len(small_list) == chunk_size: #check if small list has reached chunk size
        size_flag = 'reached_chunk_size'  
    
    return(small_list)

In [9]:
# Identify directions
def find_direction(lst):
    list_size = len(lst)
    diff = {}
    dir_flag = 'Error : not calculated'
   
    for i in range(list_size-1): #write the list into a dictionary 
        diff[i] = lst[i+1] - lst[i]       
        
    if list(diff.values())[-1] == 0:
        dir_flag = 'flat'        
        
    elif all(x>0 for x in diff.values()):
        dir_flag = 'positive'
    
    elif all(x<0 for x in diff.values()):
        dir_flag = 'negative'

    elif list(diff.values())[-2] >= 0 and list(diff.values())[-1] < 0:
        dir_flag = 'dir_change_negative'      

    elif list(diff.values())[-2] <= 0 and list(diff.values())[-1] > 0:
        dir_flag = 'dir_change_positive'      
        
    else:
        dir_flag = 'no_single_direction'
        
    return(dir_flag)

In [10]:
def write_price_dict(price_details, tick_price, truncated_tick_price, direction):
    price_details['tick_price'].append(tick_price)
    price_details['truncated_tick_price'].append(truncated_tick_price)
    price_details['direction'].append(direction)
    return(price_details)

## Historic data analysis

In [11]:
df = pd.read_csv('data/historic_data/DAT_NT_EURUSD_T_BID_202005.csv', sep = ';', header = None, names = ['timestamp', 'price', 'unknown'])
df.head()

,timestamp,price,unknown
0,20200501 000000,1.09450,0
1,20200501 000002,1.09448,0
2,20200501 000003,1.09447,0
3,20200501 000006,1.09445,0
4,20200501 000027,1.09446,0


## Code Engine

### Dummy stream tester

In [12]:
full_list = [1,2,3,2,5,6,7,8,9,10,11,12,12,12,11,10,10,9,9,8,8,7,7]

chunk_size = 3
change_position = chunk_size - 1
small_list = collections.deque([])
size_flag = 0

for i, val in enumerate(full_list):    
    small_list = get_chunks(i, val)
    if len(small_list) >= chunk_size:
        d = find_direction(small_list) 
        print(small_list,val, d)
    else:
        print(val)

1
2
deque([1, 2, 3]) 3 positive
deque([2, 3, 2]) 2 dir_change_negative
deque([3, 2, 5]) 5 dir_change_positive
deque([2, 5, 6]) 6 positive
deque([5, 6, 7]) 7 positive
deque([6, 7, 8]) 8 positive
deque([7, 8, 9]) 9 positive
deque([8, 9, 10]) 10 positive
deque([9, 10, 11]) 11 positive
deque([10, 11, 12]) 12 positive
deque([10, 11, 12]) 12 positive
deque([10, 11, 12]) 12 positive
deque([11, 12, 11]) 11 dir_change_negative
deque([12, 11, 10]) 10 negative
deque([12, 11, 10]) 10 negative
deque([11, 10, 9]) 9 negative
deque([11, 10, 9]) 9 negative
deque([10, 9, 8]) 8 negative
deque([10, 9, 8]) 8 negative
deque([9, 8, 7]) 7 negative
deque([9, 8, 7]) 7 negative


### Real Live data streamer

In [13]:
params = {
    'instruments': 'EUR_USD'
}

num = 100
chunk_size = 3
seconds_delay = 0

In [14]:
start_time = time.time()

change_position = chunk_size - 1
size_flag = 0
dir_data = {'date':0,
            'run_time':0,
             'utc-time':0,
             'instrument':'',
             'flat':0,
             'positive':0,
             'negative':0,
             'dir_change_negative':0,
             'dir_change_positive':0,
             'no_single_direction':0}


price_details = {'tick_price':[],
                 'truncated_tick_price':[],
                 'direction':[]}


small_list = collections.deque([])
dir_data['instrument'] = params['instruments']
dir_data['num'] = num
dir_data['seconds_delay'] = seconds_delay


r = pricing.PricingStream(accountID=accountID, params=params)
rv = api.request(r)

for i, resp in tqdm(enumerate(rv)):
    if i < num: # Check if we are within the required number of price iterations               
        resp_type = resp['type']        
        
        if resp_type == 'PRICE': # Check whether it is a price response                 
            dir_data['date'], dir_data['time'], time_fraction = get_date_time(resp) # Get time stamp for reference            
            sell_price, buy_price, spread, tick_price = get_prices(resp) # Get prices from the response                      
            truncated_tick_price = restrict_to_pips(tick_price) # Restrict tick price to 4 decimal places 
            small_list = get_chunks(i, truncated_tick_price) # Chunkup up the tick prices for finding direction 
            
            if len(small_list) >= chunk_size: # Check if the chunk has reached the predefined size for direction identification
                direction = find_direction(small_list) 
                dir_data[direction] += 1
                price_details = write_price_dict(price_details, tick_price, truncated_tick_price, direction)
            else: # Not yet in required chunk size
                price_details = write_price_dict(price_details, tick_price, truncated_tick_price, 'forming chunks')
        else: # Heart beat response to keep the api connection alive (Avoid timeout)
            pass

    else: # Crossed the required number of price iterations
        terminate_connection()

    time.sleep(seconds_delay)

end_time = time.time()    
seconds_elapsed = end_time - start_time
hours, rest = divmod(seconds_elapsed, 3600)
minutes, seconds = divmod(rest, 60)
dir_data['run_time'] = f'{round(hours)}:{round(minutes)}:{round(seconds)}'

winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

101it [01:42,  1.02s/it]


In [15]:
df_price_details = pd.DataFrame.from_dict(price_details)
df_price_details.to_csv('data/df_price_details.csv')
df_price_details.head()

,tick_price,truncated_tick_price,direction
0,1.129280,1.1292,forming chunks
1,1.129265,1.1292,forming chunks
2,1.129280,1.1292,flat
3,1.129290,1.1292,flat
4,1.129305,1.1293,dir_change_positive


In [16]:
df_dir_data = pd.DataFrame()
df_dir_data = df_dir_data.append(dir_data, ignore_index=True)
df_dir_data = df_dir_data[['date', 'time' ,'instrument', 'num', 'seconds_delay','run_time','positive', 'negative', 'dir_change_positive', 'dir_change_negative', 'flat', 'no_single_direction']]
df_dir_data.to_csv('data/df_dir_data.csv', mode='a', header=False)
df_dir_data.head(10)

,date,time,instrument,num,seconds_delay,run_time,positive,negative,dir_change_positive,dir_change_negative,flat,no_single_direction
0,2020-06-09,12:29:30,EUR_USD,100.0,0.0,0:1:43,22.0,0.0,37.0,18.0,2.0,0.0
